In [ ]:
import geopandas as gpd
import osmnx as ox
import folium

# Turn off warnings
import warnings
warnings.filterwarnings('ignore')

# Load boundary of Hyderabad from OpenStreetMap
hyderabad = ox.geocode_to_gdf("Hyderabad, India")

# Create a base map centered on Hyderabad
center = [hyderabad.geometry.centroid.y.values[0], hyderabad.geometry.centroid.x.values[0]]
m = folium.Map(location=center, zoom_start=12)

# Add boundary outline to the map
folium.GeoJson(hyderabad, name="Hyderabad Boundary").add_to(m)

m  # This line displays the interactive map

In [ ]:
import geopandas as gpd
import folium
import osmnx as ox
import osmnx.features as oxf  # Corrected import
import warnings
warnings.filterwarnings('ignore')

# Get Hyderabad boundary to center the map
hyderabad = ox.geocode_to_gdf("Hyderabad, India")
center = [hyderabad.geometry.centroid.y.values[0], hyderabad.geometry.centroid.x.values[0]]

# Get all bus stops from OSM
tags = {"highway": "bus_stop"}
bus_stops = oxf.features_from_place("Hyderabad, India", tags)

# Filter to keep only Point geometries
bus_stops = bus_stops[bus_stops.geometry.type == "Point"]

# Show how many bus stops were found
print(f"Found {len(bus_stops)} bus stops.")

# Create a base map
m = folium.Map(location=center, zoom_start=12)

# Add Hyderabad boundary
folium.GeoJson(hyderabad, name="Hyderabad Boundary").add_to(m)

# Add bus stops as blue dots
for _, row in bus_stops.iterrows():
    lat, lon = row.geometry.y, row.geometry.x
    folium.CircleMarker(location=[lat, lon], radius=2, color="blue", fill=True).add_to(m)

m

In [ ]:
# Add bus stops to folium map
for _, row in bus_stops.iterrows():
    lat, lon = row.geometry.y, row.geometry.x
    folium.CircleMarker(location=[lat, lon], radius=2, color="blue", fill=True).add_to(m)

m

In [ ]:
# Get road network
G = ox.graph_from_place("Hyderabad, India", network_type="drive")
edges = ox.graph_to_gdfs(G, nodes=False)

# Add roads to map
folium.GeoJson(edges, name="Roads", style_function=lambda x: {"color": "black", "weight": 0.5}).add_to(m)

m

In [ ]:
# Total length of all roads (in km)
edges["length_km"] = edges["length"] / 1000
total_length = edges["length_km"].sum()
print(f"Total road length in Hyderabad: {total_length:.2f} km")

In [ ]:
import osmnx as ox

# Download the road network for Hyderabad
G = ox.graph_from_place("Hyderabad, India", network_type="drive")

# Convert to GeoDataFrame (edges are road segments)
edges = ox.graph_to_gdfs(G, nodes=False)

# Calculate length of each road in kilometers
edges["length_km"] = edges["length"] / 1000

In [ ]:
edges = edges.to_crs(wards.crs)

In [ ]:
print(wards.columns)

In [ ]:
import geopandas as gpd

# Load GHMC ward boundaries
wards = gpd.read_file("C:\\Users\\RASMITHA\\City_mobility_dashboard_hyderabad\\data\\ghmc-wards.kml")  # update path if needed

# Reproject the road edges and wards to the same CRS (Coordinate Reference System)
edges = edges.to_crs(wards.crs)

# Spatial join: assign roads to the ward they intersect
joined = gpd.sjoin(edges, wards, how="inner", predicate="intersects")

# Group by 'Name' (the ward name), calculate total road length
road_per_ward = joined.groupby("Name")["length_km"].sum().reset_index()

# Merge road length data back into the ward polygons
wards = wards.merge(road_per_ward, on="Name", how="left")

# Fill any missing values with 0
wards["length_km"] = wards["length_km"].fillna(0)

In [ ]:
import folium
from folium.features import GeoJsonTooltip

# Create a base map centered on Hyderabad
m = folium.Map(location=[17.385, 78.4867], zoom_start=11)

# Add choropleth layer with road length per ward
folium.Choropleth(
    geo_data=wards,
    name="choropleth",
    data=wards,
    columns=["Name", "length_km"],
    key_on="feature.properties.Name",
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.3,
    legend_name="Total Road Length per Ward (km)",
    nan_fill_color="lightgrey",
).add_to(m)

# Add interactive tooltip on hover
tooltip = GeoJsonTooltip(fields=["Name", "length_km"],
                         aliases=["Ward:", "Road Length (km):"],
                         localize=True)

folium.GeoJson(
    wards,
    tooltip=tooltip,
    name="Wards"
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Display map
m